In [ ]:
import json
import pandas as pd
import addfips
import plotly.express as px
import plotly.graph_objects as go
from urllib.request import urlopen
import plotly.figure_factory as ff


af = addfips.AddFIPS()
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
#Create a dataset that has County Fips, Urbanization, political affiliation, culture group 


In [ ]:
urbanization = pd.read_csv(r"data/UrbanRual/2013Urbanization.txt", delimiter="	")
urbanization = urbanization.dropna(subset=["County"])

urbanization['State'] = urbanization.County.str.split(',').str[1]
urbanization['County'] = urbanization.County.str.split(',').str[0]
urbanization['County'] = urbanization.County.str.strip()
urbanization['State'] = urbanization.State.str.strip()
urbanization["fips"] = urbanization.apply(lambda x: af.get_county_fips(x["County"], state=x["State"]), axis=1)
urbanization = urbanization.drop(["Notes"], axis=1)
urbanization.sample(4)

In [ ]:
cultural = pd.read_excel(r"data/Cultural/woodardsCounty.xlsx")
cultural["fips"] = cultural.apply(lambda x: af.get_county_fips(x["COUNTY NAME (FULL)"], state=x["STATE NAME"]), axis=1)
cultural = cultural.rename({"WOODARD NATION NAME":"Cultural ID"}, axis=1)
cultural = cultural.drop(["COUNTY NAME (SHORT)", "COUNTY NAME (FULL)", "STATE NAME"], axis=1)
cultural

In [ ]:
final = pd.merge(cultural, urbanization, on="fips")
final.fips = final.fips.astype(str)
final.to_csv("./data/Groups/final_groups.csv")

In [ ]:
final

In [ ]:
final.dtypes

In [ ]:
final.sample(5)

In [ ]:
fig = px.choropleth(final, geojson=counties, locations='fips', color='Cultural ID',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )


In [ ]:
fig